In [ ]:
import math as m
import numpy as np
import matplotlib as mpl
from PIL import Image as im
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import sklearn.datasets as datasets
from sklearn.linear_model import LogisticRegression


TEMPFILE_NAME = "temp.png"

def save_to_gif(filename, images, duration=100):
    images[0].save(
        filename,
        optimize=False,
        save_all=True,
        append_images=images[1:],
        loop=0,
        duration=duration,
    )

np.random.seed(1)

COLORS = np.array(['#9E005D', 'green', '#0D718C'])

# custom CMAP
cvals  = [0, .5, 1]
colors = ['#0D718C', 'white', '#9E005D']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['grey', 'white', '#9E005D']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_PURPLE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['grey', 'white', 'green']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_GREEN = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['grey', 'white', '#0D718C']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_BLUE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

CENTERS = [[0, 0]]
STDEV = 1
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=CENTERS,
        cluster_std=STDEV,
        random_state=1
    )
xlim_min = CENTERS[0][0] - 3 * STDEV
xlim_max = CENTERS[0][0] + 3 * STDEV
ylim_min = CENTERS[0][1] - 3 * STDEV
ylim_max = CENTERS[0][1] + 3 * STDEV

def generate_line_data(t, w1, w2, b):
    X = np.array(list(filter(lambda x : w1 * x[0] + w2 * x[1] + b < -.5 or w1 * x[0] + w2 * x[1] + b > .5, t)))
    Y = np.array([0 if w1 * x[0] + w2 * x[1] + b >= 0 else 2 for x in X])
    return X, Y

def sigmoid(x):
    e = np.exp(-x)
    return 1 / (1 + e)

def generate_3_classes_1D(SIZE):
    X = np.linspace(0, 5 * SIZE, SIZE) + np.random.randn(SIZE) * 4
    Y = np.array([1 if x < 18 else 2 if x < 60 else 0 for x in X])
    return X, Y

def generate_3_classes_2D(t, w1, w2):
    f1 = lambda x1, x2: w1[0] * x1 + w1[1] * x2
    f2 = lambda x1, x2: w2[0] * x1 + w2[1] * x2
    X = np.array(list(filter(lambda x : (f1(x[0], x[1]) < -.5 or f1(x[0], x[1]) > .5) and (f2(x[0], x[1]) < -.5 or f2(x[0], x[1]) > .5), t)))
    Y = np.array([0 if f1(x[0], x[1]) >= 0 and f2(x[0], x[1]) >= 0 else 1 if (f1(x[0], x[1]) < 0 and f2(x[0], x[1]) >= 0) else 2 for x in X])
    return X, Y

# OTHER FUNCTIONS
def generate_circle_data(t):
    X = np.array(list(filter(lambda x : (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 < 1 or (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 > 1.5, t)))
    Y = np.array([0 if (x[0] - CENTERS[0][0])**2 + (x[1] - CENTERS[0][1])**2 >= 1 else 2 for x in X])
    return X, Y

def generate_square_data(t):
    X = np.array(list(filter(lambda x : x[0]**2 - x[1] < .4 or x[0]**2 - x[1] > 1.1, t)))
    Y = np.array([1 if x[0]**2 - x[1] >= .75 else 0 for x in X])
    return X, Y

def generate_curve_data(t):
    X = np.array(list(filter(lambda x : m.cos(4*x[0]) - x[1] < -.5 or m.cos(4*x[0]) - x[1] > .5, t)))
    Y = np.array([1 if m.cos(4*x[0]) - x[1] >= 0 else 0 for x in X])
    return X, Y

## A different perspective

In [ ]:
np.random.seed(3)
X_ages = np.random.randn(20) * 2 + 18
X_ages = np.append(X_ages, np.random.randn(20) * 2 + 30)
Y_ages = np.array([0 if x < 23 else 1 for x in X_ages])
c_ages = np.array([COLORS[0], COLORS[2]])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
ax.set_xlim(np.min(X_ages)-2, np.max(X_ages)+2)
ax.set_ylim(-0.1, 1.1)
fig.savefig('002.png')
plt.close()

## Linear function

In [ ]:
def snap(w1, w2, b):
    xx = np.linspace(-6, 6, 400)
    yy = np.linspace(-6, 6, 400)
    x, y = np.meshgrid(xx, yy)

    fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(10, -160)

    ax.scatter(X[:,0], X[:,1], w1 * X[:,0] + w2 * X[:,1] + b, color=COLORS[Y].tolist(), s=400, alpha=.4)
    ax.plot_surface(x, y, np.zeros_like(x), color='grey', alpha=.2)
    ax.set_xlabel('$x_1$', fontsize=15)
    ax.set_ylabel('$x_2$', fontsize=15)
    ax.set_title(f'{w1:.2f}' + '$x_1 + $' + f'{w2:.2f}' + '$x_2 + $' + f'{b:.2f}', fontsize=40)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_zlim(min(5 * X[:,0] + 5 * X[:,1] + 5), max(5 * X[:,0] + 5 * X[:,1] + 5))
    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []

for i in range(10, 50):
    images.append(snap(i/10, 1, 0))

for i in range(10, 50): 
    images.append(snap(5, i/10, 0))

for i in range(10, 50):
    images.append(snap(5, 5, i/10))

save_to_gif("029.gif", images, 10)

## Linear Function perspective

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(-x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(xlim_min, xlim_max, 300)
        yy = np.linspace(ylim_min, ylim_max, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -25)

        ax.plot_surface(x, y, z3, color=COLORS[2], alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x, y, z2, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color=COLORS[2], alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z1, levels=1, colors='purple')
        # ax.contour(x, y, z2, levels=1, colors='g')
        ax.contour(x, y, z3, levels=1, colors=COLORS[2])
        
        # ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '030_blue', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6159  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3975 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2704 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1881 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1473
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1098 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0868
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0755 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0580
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0501
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0438 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0432
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0416
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0328
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0359 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0298 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0281
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0296
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0281
2/

<Sequential name=sequential_54, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(-x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(xlim_min, xlim_max, 300)
        yy = np.linspace(ylim_min, ylim_max, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -25)

        # ax.plot_surface(x, y, z2, color='purple', alpha=0.3, label='Softmax(x1)')
        ax.plot_surface(x, y, z2, color=COLORS[1], alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z2, levels=1, colors='purple')
        ax.contour(x, y, z2, levels=1, colors=COLORS[1])
        # ax.contour(x, y, z3, levels=1, colors='b')
        
        # ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '030_green', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2900  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9668 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6978
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5238
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3856 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2981
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2278 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1778
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1440 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1173 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0998
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0869 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0796 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0671
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0656
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0506
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0505
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0448
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0456
2/2

<Sequential name=sequential_55, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(-x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(xlim_min, xlim_max, 300)
        yy = np.linspace(ylim_min, ylim_max, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(20, -25)

        ax.plot_surface(x, y, z1, color=COLORS[0], alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x, y, z1, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        ax.contour(x, y, z1, levels=1, colors=COLORS[0])
        # ax.contour(x, y, z1, levels=1, colors='g')
        # ax.contour(x, y, z3, levels=1, colors='b')
        
        ax.scatter(X[:,0], X[:,1], W[0][0] * X[:,0] + W[1][0] * X[:,1] + B[0], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][1] * X[:,0] + W[1][1] * X[:,1] + B[1], color=COLORS[Y].tolist(), s=200, alpha=.4)
        # ax.scatter(X[:,0], X[:,1], W[0][2] * X[:,0] + W[1][2] * X[:,1] + B[2], color=COLORS[Y].tolist(), s=200, alpha=.4)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-15, 15)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '030_purple', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.8909  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4817 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1633
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8604
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6400
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4780
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3707
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2833 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2270
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1797 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1484 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1279 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1065 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0934 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0844 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0778
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0682 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0590
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0568
2

<Sequential name=sequential_56, built=True>

In [ ]:
def softmax(x1, x2):
    exp_x1 = np.exp(x1)
    exp_x2 = np.exp(x2)
    total = 1 + exp_x1 + exp_x2
    return exp_x1 / total, exp_x2 / total, 1 / total

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xx = np.linspace(xlim_min, xlim_max, 300)
        yy = np.linspace(ylim_min, ylim_max, 300)
        x, y = np.meshgrid(xx, yy)

        z1, z2, z3 = softmax(x, y)

        W = self.model.layers[0].get_weights()[0]
        B = self.model.layers[0].get_weights()[1]

        fig = plt.figure(figsize =(14, 9), constrained_layout=True, facecolor=None)
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(25, -75)

        # ax.plot_surface(x, y, z2, color='purple', alpha=0.3, label='Softmax(x1)')
        # ax.plot_surface(x_z1, y_z1, z1, color='g', alpha=0.3, label='Softmax(x2)')
        # ax.plot_surface(x, y, z3, color='b', alpha=0.3, label='Softmax(x2)')
        # ax.contour(x, y, z2, levels=1, colors='purple')
        # ax.contour(x, y, z1, levels=1, colors='g')
        # ax.contour(x, y, z3, levels=1, colors='b')
        meshData = np.c_[x.ravel(), y.ravel()]
        ax.contourf(x, y, self.model.predict(meshData)[:, 0].reshape(x.shape), cmap=CMAP_PURPLE, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)
        ax.contourf(x, y, self.model.predict(meshData)[:, 1].reshape(x.shape), cmap=CMAP_GREEN, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)
        ax.contourf(x, y, self.model.predict(meshData)[:, 2].reshape(x.shape), cmap=CMAP_BLUE, alpha=.4, levels=np.linspace(0.55, 1, 10), zorder=2, offset=-1)

        purple = np.where(Y==0)
        X_purple, Y_purple = X[purple], Y[purple]
        green = np.where(Y==1)
        X_green, Y_green = X[green], Y[green]
        blue = np.where(Y==2)
        X_blue, Y_blue = X[blue], Y[blue]
        
        ax.scatter(X_purple[:,0], X_purple[:,1], W[0][0] * X_purple[:,0] + W[1][0] * X_purple[:,1] + B[0], color=COLORS[Y_purple].tolist(), s=200, alpha=.5, zorder=0)
        ax.scatter(X_green[:,0], X_green[:,1], W[0][1] * X_green[:,0] + W[1][1] * X_green[:,1] + B[1], color=COLORS[Y_green].tolist(), s=200, alpha=.5, zorder=0)
        ax.scatter(X_blue[:,0], X_blue[:,1], W[0][2] * X_blue[:,0] + W[1][2] * X_blue[:,1] + B[2], color=COLORS[Y_blue].tolist(), s=200, alpha=.5, zorder=0)

        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        ax.set_zlim(-1, 6)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '030_all', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6154  
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 164us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3997
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 173us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2936
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2279
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 1s 177us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1694
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 161us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 161us/step
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 161us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1316
2813/2813 ━

<Sequential name=sequential_57, built=True>

## 3 classes 2D training

In [ ]:
cvals  = [0, .5, 1]
colors = ['white', 'white', '#9E005D']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_PURPLE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', 'green']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_GREEN = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

cvals  = [0, .5, 1]
colors = ['white', 'white', '#0D718C']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
CMAP_BLUE = mpl.colors.LinearSegmentedColormap.from_list("", tuples, 100)

class LR():

    def __init__(self, model):
        """
        Parameters:
            model: A keras model.
                Since our class is called DenseGraph(), the keras model should only contain dense layers.
        """
        self.model = model

    def _snap_learning_3D(self, X, Y, filename, rot):
        """
        Take snapshot of input with decision boundary
        """
        xplot = np.linspace(-3, 3, 100)
        x, y = np.meshgrid(xplot, xplot)
        meshData = np.c_[x.ravel(), y.ravel()]
        fig, ax = plt.subplots()
        ax.contourf(x, y, self.model.predict(meshData)[:, 0].reshape(x.shape), cmap=CMAP_PURPLE, alpha=.4)
        ax.contourf(x, y, self.model.predict(meshData)[:, 1].reshape(x.shape), cmap=CMAP_GREEN, alpha=.4)
        ax.contourf(x, y, self.model.predict(meshData)[:, 2].reshape(x.shape), cmap=CMAP_BLUE, alpha=.4)
        ax.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=400, alpha=.5)
        ax.set_xlim(xlim_min, xlim_max)
        ax.set_ylim(ylim_min, ylim_max)
        fig.savefig(filename + '.png')
        plt.close()

        return np.asarray(im.open(filename + '.png'))


    def animate_learning_3D(self, X, Y, snap_freq=10, filename='learn', duration=1000, **kwargs):
        """
        Make GIF from snapshots of decision boundary at given snap_freq

        Parameters:
            X : ndarray
                input to a Keras model
            Y : ndarray
                classes to be learned
            snap_freq : int
                number of epochs after which to take a snapshot
            filename : str
                name of file to save as GIF
            duration : int
                duration in ms between images in GIF
            **kwargs : other params
                paramter inputs to model.fit

        Returns:
            The model after learning
        """

        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for i in range(int(epochs / snap_freq)):
            self.model.fit(X, keras.utils.to_categorical(Y), epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning_3D(X, Y, filename, i)))

        images[0].save(
            filename + '.gif',
            optimize=False,  # important for transparent background
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning_3D(X, Y, 1, '031', 200, epochs=40, batch_size=100)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.5341  
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 185us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 165us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1201
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - loss: 0.8286
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6127
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 166us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 174us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4455
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 164us/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3373
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/s

<Sequential name=sequential_58, built=True>

## Concentric circles

In [ ]:
DATA, _ = datasets.make_blobs(
        n_samples=200,
        centers=[[-1, 1]],
        cluster_std=1,
        random_state=1
    )
xlim_min = CENTERS[0][0] - 3 * STDEV
xlim_max = CENTERS[0][0] + 3 * STDEV
ylim_min = CENTERS[0][1] - 3 * STDEV
ylim_max = CENTERS[0][1] + 3 * STDEV

X, Y = generate_circle_data(DATA)

plt.scatter(X[:,0], X[:,1], color=COLORS[Y].tolist(), s=100, alpha=.9)
plt.xlabel("$x_1$", fontsize=15)
plt.ylabel("$x_2$", fontsize=15)
plt.savefig("032.png")
plt.close()

## Neural Network

In [ ]:
class LR():

    def __init__(self, model):
        self.model = model

    def _snap_learning(self, X, Y, filename):
        h = .02
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                             np.arange(y_min, y_max, h))
        meshData = np.c_[xx.ravel(), yy.ravel()]

        fig, ax = plt.subplots(frameon=False)
        ax.scatter(X[:, 0], X[:, 1], color=COLORS[Y].tolist(), s=100, alpha=.9)
        Z = self.model.predict(meshData)
        Z = np.array([z[0] for z in Z]).reshape(xx.shape)
        ax.contourf(xx, yy, Z, alpha=.4, cmap=CMAP)
        fig.savefig(filename + '.png')
        plt.close()
        return np.asarray(im.open(filename + '.png'))

    def animate_learning(self, X, Y_0_1, snap_freq=10, filename='learn', duration=1000, **kwargs):
        images = []
        if 'epochs' in kwargs:
            epochs = kwargs['epochs']
            kwargs.pop('epochs', None)
        else:
            epochs = snap_freq

        for _ in range(int(epochs / snap_freq)):
            self.model.fit(X, Y_0_1, epochs=snap_freq, **kwargs)
            images.append(im.fromarray(self._snap_learning(X, Y, filename)))

        images[0].save(
            filename + '.gif',
            optimize=False,
            save_all=True,
            append_images=images[1:],
            loop=0,
            duration=duration
        )
        return self.model

Y_0_1 = Y.copy()
Y_0_1[np.where(Y_0_1 == 0)] = 1
Y_0_1[np.where(Y_0_1 == 2)] = 0

model = keras.models.Sequential()
model.add(keras.layers.Dense(3, input_dim=2, activation="tanh"))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-1))

obj = LR(model)
obj.animate_learning(X, Y_0_1, 1, '033', 100, epochs=50, batch_size=100)

/Users/lance/Documents/BostonUniversity/Data-Science-Fundamentals/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6617  
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 163us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6168
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 161us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5970
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 161us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5755
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5640
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 173us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5596
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5401
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5115
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 161us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5068
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 162us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4975
3335/3335 ━━━━━━━━━━━━━━━━━━━━ 1s 303us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 

<Sequential name=sequential_59, built=True>

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
ax.set_xlim(np.min(X_ages)-2, np.max(X_ages)+2)
ax.set_ylim(-0.1, 1.1)
ax.axvline(23, color=COLORS[2], linestyle='dashed')
fig.savefig('003.png')
plt.close()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
ax.set_xlim(-8, np.max(X_ages)+2)
ax.set_ylim(-0.1, 1.1)
ax.set_title('Legal Drinking Age', fontsize=20)
ax.set_xlabel('X', fontsize=20)
fig.savefig('001.png')
plt.close()

## Side by side

In [ ]:
fig, ax2 = plt.subplots()
ax2.scatter(X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
ax2.set_xlim(-8, np.max(X_ages)+2)
ax2.set_ylim(-0.1, 1.1)
ax2.set_title('Shift Data', fontsize=20)
ax2.set_xlabel('X', fontsize=20)

fig.savefig('002.png')
plt.close()

## With Sigmoid

In [ ]:
xplot = np.linspace(-np.max(X_ages)-2, np.max(X_ages)+2, 300)
fig, ax2 = plt.subplots()
ax2.scatter(X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
ax2.plot(xplot, sigmoid(xplot), color='maroon', linewidth=4, label='$\\sigma(X)$')
ax2.axvline(0, color='grey', linestyle='dotted', linewidth=2)
ax2.set_xlim(-8, np.max(X_ages)+2)
ax2.set_ylim(-0.1, 1.1)
ax2.set_title('Shift Data', fontsize=20)
ax2.set_xlabel('X', fontsize=20)
ax2.legend(loc='upper left', fontsize=20)

fig.savefig('003.png')
plt.close()

## Shifting Data

In [ ]:
def snap_shift(b):
    xplot = np.linspace(-np.max(X_ages)-2, np.max(X_ages)+2 + b, 300)

    fig, ax2 = plt.subplots()
    shifted_X_ages = X_ages.copy()
    shifted_X_ages -= b
    ax2.scatter(shifted_X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
    ax2.plot(xplot - b, sigmoid(xplot - b), color='maroon', linewidth=4, label='$\\sigma(X - b)$'.replace('b', str(b)))
    ax2.axvline(0, color='grey', linestyle='dotted', linewidth=2)
    ax2.set_xlim(-8, np.max(X_ages)+2)
    ax2.set_ylim(-0.1, 1.1)
    ax2.set_title('Shift Data', fontsize=20)
    ax2.set_xlabel('X - b'.replace('b', str(b)), fontsize=20)
    ax2.legend(loc='upper left', fontsize=20)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []
for i in range(220):
    images.append(snap_shift(i/10))

save_to_gif('004.gif', images, duration=30)

## Scaling Data

In [ ]:
def snap_scale(w):
    b = 21
    xplot = np.linspace(-np.max(X_ages)-2, np.max(X_ages)+2 + b, 1000)

    fig, ax2 = plt.subplots()
    scaled_X_ages = X_ages.copy()
    scaled_X_ages = w * (scaled_X_ages - b)
    ax2.scatter(scaled_X_ages, np.zeros_like(X_ages), color=c_ages[Y_ages].tolist(), s=300, alpha=.4)
    ax2.plot(w* (xplot - b), sigmoid(w * (xplot - b)), color='maroon', linewidth=4, label='$\\sigma(w * X - b)$'.replace('b', f'{w * b:.1e}').replace('w', f'{w:.2f}'))
    ax2.axvline(0, color='grey', linestyle='dotted', linewidth=2)
    ax2.set_xlim(-20, 10)
    ax2.set_ylim(-0.1, 1.1)
    ax2.set_title('Scaled Data', fontsize=20)
    ax2.set_xlabel('w * X - b'.replace('b', f'{w * b:.1e}').replace('w', f'{w:.2f}'), fontsize=20)
    ax2.legend(loc='upper left', fontsize=10)

    fig.savefig(TEMPFILE_NAME)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE_NAME)))

images = []
for i in range(10, 500):
    images.append(snap_scale(i/100))

save_to_gif('005.gif', images, duration=30)